# Zenodo Toolbox: Database Integration (SQLite)

This notebook demonstrates how to integrate a SQLite database with the Zenodo Toolbox with a preset structure. We'll use the database to store and manage information about Zenodo records, making it easier to track and query our data submissions. Most database operations are handled automatically by those functions that are interacting with Zenodo, but in order to provide some knowledge to customize the toolbox for individual projects, this Notebooks helps to understand the underlying processes.

## Database Configuration

We're using SQLite as our database engine. The configuration is defined in a YAML file, which specifies:

- Database type (local SQLite)
- Database file path
- Table structures
- Indexes and foreign key relationships

Key aspects of our database structure:

1. **Records**: Stores main record information (concept_recid, title, DOI, etc.)
2. **Links**: Contains various URLs associated with each record
3. **Mainfiles**: Tracks files uploaded to each record
4. **States**: Keeps track of availability status for different record components
5. **Thumbnails**: Stores links to thumbnail images at various resolutions
6. **Responses**: Saves raw JSON responses from Zenodo API calls
7. **Rate Limiter**: Helps manage API request frequency
8. **Operations**: Logs operations performed on records

## Database Integration Steps

1. **Initialize Database Connection**: We'll create a connection to our SQLite database using the configuration from our YAML file.

2. **Create Tables**: If they don't exist, we'll create the necessary tables as defined in our configuration.

3. **Insert/Update Records**: As we interact with Zenodo (creating drafts, uploading files, publishing records), we'll store the relevant information in our database.

4. **Query Data**: We'll demonstrate how to retrieve information from our database, allowing for easy tracking and management of our Zenodo submissions.

## Benefits of Database Integration

- **Decentralized Data Access**: Maintain a local record of all Zenodo interactions
- **Efficient Querying**: Quickly find and analyze your Zenodo submissions
- **Version Tracking**: Keep track of the status and versions of each record and its components
- **Performance**: Reduce the need for frequent API calls by storing data locally

Let's start by initializing our database connection and creating the necessary tables.


In [ ]:
import os
from pathlib import Path

os.chdir(Path().absolute().parent) if Path().absolute().name == "Tutorials" else None
from utilities import load_config, load_json
from db_tools import clear_operations_by_status, get_row, initialize_db, print_table, upsert_operation, upsert_published_data

# Load DB configuration and change path to DB
db_config = load_config("Configs/db_config.yaml")
db_path = "Tutorials/sandbox.db"
db_config["local_db_path"] = db_path

print(f"Configuration loaded. DB will be initialized at: {db_path}")

## Initialize Database Connection

Now, let's initialize our database connection. This function will create the database if it doesn't exist, or connect to an existing one.

In [ ]:
db_connection = initialize_db(db_config)
if db_connection:
    print("Database connection initialized successfully.")
else:
    print("Failed to initialize database.")


This should have created a database at the path given in db_path.

## Database Operations

Let's demonstrate some common database operations using our custom functions.

### 1. Inserting a New Operation


In [ ]:
zenodo_response_data = load_json("Tutorials/Output/sandbox_drafts.json")[-1] # catch the latest response data

operation_result = upsert_operation(db_connection, "new_record_created", zenodo_response_data)
print(f"Operation insert successful: {operation_result}")


### 2. Updating Published Data


In [ ]:
from utilities import get_thumbnails, is_edm_available, is_metsmods_available

file_response_data = load_json("Tutorials/Output/sandbox_files.json")[-1]
thumbnails_data = get_thumbnails(file_response_data)
additional_data = {
    "type": "image",
    "subset": "project_sandbox",
    "changelogs": {},
    "filedata": file_response_data,
    "thumbnails_data": thumbnails_data,
    "edm_available": is_edm_available(file_response_data),
    "metsmods_available": is_metsmods_available(file_response_data),
    "thumbnails_available": True if thumbnails_data else False,
}

published_data = load_json("Tutorials/Output/sandbox_published.json")[-1]

publish_result = upsert_published_data(db_connection, published_data, additional_data)
print(f"Publish data upsert successful: {publish_result}")


### 3. View Database Entries of each Table

Let's have a look at what has been written into the database so far:

In [ ]:
for table_name in [list(i.keys())[0] for i in db_config["db_structures"]["tables"]]:
    print(f"{table_name}:")
    print_table(db_connection, table_name)

### 4. Update existing Operation

As you can see in the last table "operations", we did not update it as published, it still remains with the status "draft". If we use this function, we can update the status:

In [ ]:
operation_result = upsert_operation(db_connection, "published", published_data)
print(f"Operation insert successful: {operation_result}")

print_table(db_connection, "operations")

### 5. Clearing Published Operations

Now that the status is "published", it is not necessary to keep it in the current operations table, so we can clear it by providing the Concept Record IDs:

In [ ]:
clear_result = clear_operations_by_status(db_connection, "published", [published_data["conceptdoi"]])
print(f"Clear published operations result: {clear_result}")

print_table(db_connection, "operations")

If you want to clear all entries with the status 'published', just leave out the list with Concept Record IDs:

In [ ]:
clear_result = clear_operations_by_status(db_connection, "published")
print(f"Clear published operations result: {clear_result}")

print_table(db_connection, "operations")

<small>
Note: Change "published" to "draft" if you want to remove the drafts from operations table.
</small>

## Query Results

Now we can retrieve some essential data from the local database in our preferred structure instead of querying Zenodo.

<small>

Note: Querying your uploaded Zenodo records seems to be limited to 10.000 records (20.000 with a reverse trick).

</small>

Let's start with retrieving the persistent direct link for the test_image_2.png:

In [ ]:
published_data = load_json("Tutorials/Output/sandbox_published.json")[-1]
concept_recid = published_data["conceptrecid"]
row_data = get_row(db_connection, "mainfiles", "concept_recid", concept_recid, print_result=True)
print(f'Persistent Direct Link to {row_data["filename"]}: {row_data["direct_link"]}')

## Closing the Database Connection

Always remember to close the database connection when you're done.


In [ ]:
db_connection.close()
print("Database connection closed.")


This notebook demonstrates the basic operations of creating a database connection, inserting and updating records, and querying the database. In a real-world scenario, you would integrate these operations with your Zenodo API calls and file processing logic.

Remember to handle exceptions and add more detailed error checking in a production environment. Also, consider implementing connection pooling and more advanced querying techniques for larger-scale applications.
